In [1]:
import pandas_datareader.data as web
import datetime

start = datetime.datetime(1986, 1, 1)
end = datetime.datetime(2025, 6, 18)

fed = web.DataReader('FEDFUNDS', 'fred', start, end)
brent = web.DataReader('POILBREUSDM', 'fred', start, end)
fao = web.DataReader('PFOODINDEXM', 'fred', start, end)  # FAO Food Price Index


In [2]:
for i in [fed, brent, fao]:
    print(i.head(1))

            FEDFUNDS
DATE                
1986-01-01      8.14
            POILBREUSDM
DATE                   
1990-01-01     20.98913
            PFOODINDEXM
DATE                   
1992-01-01    57.735817


In [3]:
#create three lags for each series and put in dataframe 
import pandas as pd
def create_lags(series, lags=3):
    """Create lagged features for a given series."""
    return pd.concat([series.shift(i) for i in range(0, lags + 1)], axis=1)

# Create lagged features for each series
fed_lags = create_lags(fed['FEDFUNDS'])
brent_lags = create_lags(brent['POILBREUSDM'])
fao_lags = create_lags(fao['PFOODINDEXM'])

# Combine all lagged features into a single DataFrame
lagged_data = pd.concat([fed_lags, brent_lags, fao_lags], axis=1)

# Rename columns for clarity
# lagged_data.columns = [f'{col}_lag{i+1}' for col in lagged_data.columns for i in range(3)]
# Drop rows with NaN values (due to lagging)
lagged_data.dropna(inplace=True)
# Rename columns for clarity
lagged_data.columns = [
    f'FEDFUNDS(t-{i})' for i in range(4)
] + [f'POILBREUSDM(t-{i})' for i in range(4)] + [f'PFOODINDEXM(t-{i})' for i in range(4)]

# Display the first few rows of the lagged data
lagged_data.tail()

,FEDFUNDS(t-0),FEDFUNDS(t-1),FEDFUNDS(t-2),FEDFUNDS(t-3),POILBREUSDM(t-0),POILBREUSDM(t-1),POILBREUSDM(t-2),POILBREUSDM(t-3),PFOODINDEXM(t-0),PFOODINDEXM(t-1),PFOODINDEXM(t-2),PFOODINDEXM(t-3)
DATE,,,,,,,,,,,,
2024-12-01,4.48,4.64,4.83,5.13,73.227727,73.508571,75.266522,73.057619,132.224070,133.314689,132.702539,130.718539
2025-01-01,4.33,4.48,4.64,4.83,78.193913,73.227727,73.508571,75.266522,135.640085,132.224070,133.314689,132.702539
2025-02-01,4.33,4.33,4.48,4.64,75.193000,78.193913,73.227727,73.508571,132.066798,135.640085,132.224070,133.314689
2025-03-01,4.33,4.33,4.33,4.48,71.741429,75.193000,78.193913,73.227727,128.966254,132.066798,135.640085,132.224070
2025-04-01,4.33,4.33,4.33,4.33,66.927273,71.741429,75.193000,78.193913,130.158567,128.966254,132.066798,135.640085


In [4]:
g = lagged_data.columns[1]

lagged_data.columns.str.contains(f'\\(t\\-')

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [9]:
brent_lags.tail()

,POILBREUSDM,POILBREUSDM,POILBREUSDM,POILBREUSDM
DATE,,,,
2024-12-01,73.227727,73.508571,75.266522,73.057619
2025-01-01,78.193913,73.227727,73.508571,75.266522
2025-02-01,75.193000,78.193913,73.227727,73.508571
2025-03-01,71.741429,75.193000,78.193913,73.227727
2025-04-01,66.927273,71.741429,75.193000,78.193913


In [6]:
#save dataframe to excel file
lagged_data.to_excel('exog_lagged_data.xlsx', index=True)



In [8]:
import pandas as pd

# Placeholder URLs (update manually if needed)
imf_weo_oil_url = "https://www.imf.org/-/media/Files/Publications/WEO/2024/April/English/data/WEOApr2024all.ashx"
oecd_fao_outlook_url = "https://www.agri-outlook.org/download/en/OECD-FAO_Agricultural_Outlook_2024.xlsx"
fomc_projection_url = "https://www.federalreserve.gov/monetarypolicy/fomcprojtabl20240612.htm"

# Create a dataframe with the key links and guidance
projections_sources = pd.DataFrame({
    "Variable": ["Brent Crude Oil", "FAO Food Index", "Federal Funds Rate"],
    "Source": ["IMF World Economic Outlook", "OECD-FAO Agricultural Outlook", "US Federal Reserve (FOMC Dot Plot)"],
    "Direct Forecast?": ["✅", "🟡 (via commodity categories)", "🟡 (not time series)"],
    "Frequency": ["Annual", "Annual (10-year)", "Year-end levels"],
    "Format": ["Excel (IMF)", "Excel (OECD)", "HTML Table (PDF also)"],
    "Download URL": [imf_weo_oil_url, oecd_fao_outlook_url, fomc_projection_url],
    "Notes": [
        "Look for APSP (Average Petroleum Spot Price) under commodity tables",
        "Use cereal/oil/sugar price outlooks as proxies for FAO food index",
        "Check latest FOMC Summary of Economic Projections; parse manually"
    ]
})


#print ace_tools version 
projections_sources

,Variable,Source,Direct Forecast?,Frequency,Format,Download URL,Notes
0,Brent Crude Oil,IMF World Economic Outlook,✅,Annual,Excel (IMF),https://www.imf.org/-/media/Files/Publications...,Look for APSP (Average Petroleum Spot Price) u...
1,FAO Food Index,OECD-FAO Agricultural Outlook,🟡 (via commodity categories),Annual (10-year),Excel (OECD),https://www.agri-outlook.org/download/en/OECD-...,Use cereal/oil/sugar price outlooks as proxies...
2,Federal Funds Rate,US Federal Reserve (FOMC Dot Plot),🟡 (not time series),Year-end levels,HTML Table (PDF also),https://www.federalreserve.gov/monetarypolicy/...,Check latest FOMC Summary of Economic Projecti...
